## calculate the Metabolic Index for each grid point at each depth for FOSI hindcast

In [1]:
import metabolic as mi
import xarray as xr
import numpy as np
import netCDF4
import os
import warnings
warnings.filterwarnings('ignore')
import matplotlib.colors as colors
from matplotlib import cm
import matplotlib.pyplot as plt

In [2]:
USER = os.environ['USER']
path = f'/glade/scratch/{USER}/DPLE-results'
a1,a2,b1,b2 = [240,380,125,320]
#tres = 'monthly'
tres = 'yearly'

In [4]:
filename = f'{path}/DPLE_driftcorrected_TEMP_ens_mean_{tres}.nc'
ds1 = xr.open_dataset(filename)
Temp = ds1['TEMP']
#
filename = f'{path}/DPLE_driftcorrected_pO2_ens_mean_{tres}.nc'
ds2 = xr.open_dataset(filename)
pO2 = ds2['pO2']
Temp

<xarray.DataArray 'TEMP' (Y: 64, year: 10, z_t: 35, nlat: 140, nlon: 195)>
[611520000 values with dtype=float64]
Coordinates:
  * Y        (Y) int64 1954 1955 1956 1957 1958 ... 2013 2014 2015 2016 2017
  * year     (year) int64 0 1 2 3 4 5 6 7 8 9
  * z_t      (z_t) float32 500.0 1.5e+03 2.5e+03 ... 5.277e+04 5.794e+04
  * nlat     (nlat) int64 0 1 2 3 4 5 6 7 8 ... 132 133 134 135 136 137 138 139
  * nlon     (nlon) int64 0 1 2 3 4 5 6 7 8 ... 187 188 189 190 191 192 193 194

## Calculate Φ using 13 different Eo values and same Ac value

In [9]:
Ac_dist = [10]
Eo_dist = np.arange(-.2,1.1,.1)
print(Ac_dist)
print(Eo_dist)

[10]
[-0.2 -0.1  0.   0.1  0.2  0.3  0.4  0.5  0.6  0.7  0.8  0.9  1. ]


In [10]:
%%time
nrow = len(Ac_dist)
ncol = len(Eo_dist)
from itertools import product
dss = xr.Dataset()

for i, j in product(range(nrow), range(ncol)):
    Ac = Ac_dist[i]
    Eo = Eo_dist[j]
    print(Ac,Eo)
    dss[f'Phi_{nrow*i+j}'] = mi.Phi(pO2, Temp, Ac, Eo, dEodT=0.022)
    # = xr_reshape(ppphi,'time',['year','mon'],[np.arange(70),np.arange(12)]).mean(dim='mon')

10 -0.2
10 -0.1
10 0.0
10 0.10000000000000003
10 0.2
10 0.3
10 0.4000000000000001
10 0.5
10 0.6000000000000001
10 0.7
10 0.8
10 0.9000000000000001
10 1.0000000000000002
CPU times: user 1min 38s, sys: 2min 3s, total: 3min 42s
Wall time: 3min 56s


In [11]:
dout = f'/glade/scratch/{USER}/DPLE-results'
os.makedirs(dout, exist_ok=True)
dss.load()

<xarray.Dataset>
Dimensions:  (Y: 64, year: 10, z_t: 35, nlat: 140, nlon: 195)
Coordinates:
  * Y        (Y) int64 1954 1955 1956 1957 1958 ... 2013 2014 2015 2016 2017
  * year     (year) int64 0 1 2 3 4 5 6 7 8 9
  * z_t      (z_t) float32 500.0 1.5e+03 2.5e+03 ... 5.277e+04 5.794e+04
  * nlat     (nlat) int64 0 1 2 3 4 5 6 7 8 ... 132 133 134 135 136 137 138 139
  * nlon     (nlon) int64 0 1 2 3 4 5 6 7 8 ... 187 188 189 190 191 192 193 194
Data variables: (12/13)
    Phi_0    (Y, year, z_t, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    Phi_1    (Y, year, z_t, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    Phi_2    (Y, year, z_t, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    Phi_3    (Y, year, z_t, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    Phi_4    (Y, year, z_t, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    Phi_5    (Y, year, z_t, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    ...       ...
    Phi_7    (Y, year, z_t, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    Phi_8    (Y, year, z_t, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    Phi_9    (Y, year, z_t, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    Phi_10   (Y, year, z_t, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    Phi_11   (Y, year, z_t, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    Phi_12   (Y, year, z_t, nlat, nlon) float64 nan nan nan nan ... nan nan nan

In [12]:
%%time
dss.to_netcdf(f'{dout}/DPLE_driftcorrected_Phi_Eo_space_ens_mean_{tres}.nc', mode='w')

CPU times: user 190 ms, sys: 12.7 s, total: 12.9 s
Wall time: 14.9 s
